In [1]:
#We load the enriched Apple dataset created earlier, which includes technical indicators like SMA, RSI, and Bollinger Bands. This ensures all strategies can reference the same consistent dataset.

import pandas as pd

data = pd.read_csv("../data/AAPL_enriched.csv", index_col=0, parse_dates=True)
data.tail(3)


,Open,High,Low,Close,Volume,SMA_10,SMA_50,SMA_200,Volatility_30,RSI_14,BB_MID,BB_UPPER,BB_LOWER,ATR_14
Price,,,,,,,,,,,,,,
2024-12-26,257.276679,259.179926,256.718662,258.103729,27237100.0,251.649666,235.495564,211.097456,0.008575,76.452850,246.616033,259.723060,233.509006,4.107561
2024-12-27,256.917949,257.784897,252.164833,254.685883,42355300.0,252.409964,235.975156,211.521336,0.009190,67.626275,247.645379,259.813298,235.477460,4.350269
2024-12-30,251.337769,252.603281,249.863009,251.307877,35557500.0,252.815524,236.379822,211.919042,0.009546,60.225586,248.386249,259.448570,237.323927,4.304004


In [2]:
#This helper computes performance metrics (CAGR, Volatility, Sharpe, Max Drawdown). We will reuse it for every strategy so results are directly comparable.

import numpy as np

def compute_metrics(df, label="Strategy"):
    strat_curve = (1 + df["Strategy_Return"]).cumprod()
    years = (df.index[-1] - df.index[0]).days / 365.25
    cagr = strat_curve.iloc[-1]**(1/years) - 1
    vol = df["Strategy_Return"].std() * np.sqrt(252)
    sharpe = (df["Strategy_Return"].mean() * 252) / (df["Strategy_Return"].std() * np.sqrt(252))
    roll_max = strat_curve.cummax()
    dd = (strat_curve / roll_max - 1).min()
    return {"Strategy": label, "CAGR": cagr, "Volatility": vol, "Sharpe": sharpe, "MaxDD": dd}


In [3]:
#Strategy logic: Buy when short-term SMA (10-day) crosses above long-term SMA (50-day). Sell when it crosses below. This is one of the most common trend-following rules.

sma = data.copy()
sma["Signal"] = 0
sma.loc[sma["SMA_10"] > sma["SMA_50"], "Signal"] = 1
sma.loc[sma["SMA_10"] < sma["SMA_50"], "Signal"] = -1
sma["Position"] = sma["Signal"].shift(1).fillna(0)
sma["Return"] = sma["Close"].pct_change()
sma["Strategy_Return"] = sma["Position"] * sma["Return"]

metrics_sma = compute_metrics(sma, "SMA Crossover")



In [4]:
#Strategy logic: Buy when RSI < 30 (oversold). Sell when RSI > 70 (overbought). This is a classic mean-reversion approach.

rsi = data.copy()
rsi["Signal"] = 0
rsi.loc[rsi["RSI_14"] < 30, "Signal"] = 1
rsi.loc[rsi["RSI_14"] > 70, "Signal"] = -1
rsi["Position"] = rsi["Signal"].shift(1).fillna(0)
rsi["Return"] = rsi["Close"].pct_change()
rsi["Strategy_Return"] = rsi["Position"] * rsi["Return"]

metrics_rsi = compute_metrics(rsi, "RSI Strategy")


In [5]:
#Strategy logic: Buy when price closes above the upper band (momentum breakout). Sell when it closes below the lower band (downside breakout).

bb = data.copy()
bb["Signal"] = 0
bb.loc[bb["Close"] > bb["BB_UPPER"], "Signal"] = 1
bb.loc[bb["Close"] < bb["BB_LOWER"], "Signal"] = -1
bb["Position"] = bb["Signal"].shift(1).fillna(0)
bb["Return"] = bb["Close"].pct_change()
bb["Strategy_Return"] = bb["Position"] * bb["Return"]

metrics_bb = compute_metrics(bb, "Bollinger Breakout")


In [6]:
#We now gather results from all three strategies into one table for side-by-side comparison. This makes it easy to identify which approach performs best under different metrics.

results = pd.DataFrame([metrics_sma, metrics_rsi, metrics_bb])
results



,Strategy,CAGR,Volatility,Sharpe,MaxDD
0,SMA Crossover,0.045375,0.164331,0.354689,-0.086713
1,RSI Strategy,-0.126123,0.082338,-1.620564,-0.043275
2,Bollinger Breakout,0.139849,0.044243,3.024177,-0.007036


In [7]:
#Results are saved to the logs folder so we keep an audit trail of performance across multiple strategies.

import os
from datetime import datetime

os.makedirs("../logs", exist_ok=True)
results.to_csv("../logs/Day8_strategy_comparison.csv", index=False)

with open("../logs/Day8_summary.txt", "w") as f:
    f.write(f"Day 8 run ({datetime.today().date()})\n")
    f.write(results.to_string())
